In [1]:
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [3]:
# Load using pickle
with open('IowaDataset/processed/theta.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
    data = loaded_data['data']
    labels = loaded_data['labels']

In [4]:
data.shape

(3360, 29)

In [163]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [8]:
# Split data into train and test sets
def train_test_svm(data, labels, test_size=0.1, random_state=101):
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=test_size, random_state=random_state)
    print(len(X_train))
    print(len(X_test))

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize and train the SVM model
    svm_model = SVC(kernel='linear',class_weight='balanced',probability=True)
    svm_model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = svm_model.predict(X_test_scaled)

    # Predict probabilities for the test set
    y_prob = svm_model.predict_proba(X_test)[:, 1]  # Probability of the positive class
    # Calculate the ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_prob)


    # Output performance metrics
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

    print("Predictions:", np.unique(y_pred, return_counts=True))

    scores = cross_val_score(svm_model, X_train_scaled, y_train, cv=5, scoring='f1')
    print("Cross-validation F1 scores for class 1:", scores)

    return tpr,fpr


In [9]:
# Train and evaluate SVM model
tpr,fpr=train_test_svm(data, labels) # 0.8440860215053764

3024
336
Accuracy: 0.8273809523809523
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.93      0.85       173
           1       0.91      0.72      0.80       163

    accuracy                           0.83       336
   macro avg       0.84      0.82      0.82       336
weighted avg       0.84      0.83      0.83       336

Predictions: (array([0, 1]), array([207, 129], dtype=int64))
Cross-validation F1 scores for class 1: [0.80427046 0.78467153 0.81978799 0.77757685 0.80071813]


In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_curve, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [6]:
def svm_kfold(data, labels, k=5):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=101)
    
    accuracies = []
    sensitivities = []
    specificities = []
    
    fold = 1
    for train_index, test_index in skf.split(data, labels):
        print(f"Fold {fold}")
        fold += 1

        # Split the data into train and test
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        # Scaling
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Initialize and train the SVM model
        svm_model = SVC(kernel='linear', class_weight='balanced', probability=True)
        svm_model.fit(X_train_scaled, y_train)
        
        # Make predictions
        y_pred = svm_model.predict(X_test_scaled)

        # Calculate accuracy
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)
        
        # Confusion matrix to calculate sensitivity and specificity
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)

        sensitivities.append(sensitivity)
        specificities.append(specificity)

        # Output for the current fold
        print(f"Accuracy: {acc:.4f}")
        print(f"Sensitivity (Recall): {sensitivity:.4f}")
        print(f"Specificity: {specificity:.4f}")
        print()

    # Output overall metrics
    print("=== Overall Metrics ===")
    print(f"Mean Accuracy: {np.mean(accuracies):.4f}")
    print(f"Mean Sensitivity: {np.mean(sensitivities):.4f}")
    print(f"Mean Specificity: {np.mean(specificities):.4f}")

    return np.mean(accuracies), np.mean(sensitivities), np.mean(specificities)

In [7]:
svm_kfold(data,labels)

Fold 1
Accuracy: 0.8348
Sensitivity (Recall): 0.7649
Specificity: 0.9048

Fold 2
Accuracy: 0.8021
Sensitivity (Recall): 0.7232
Specificity: 0.8810

Fold 3
Accuracy: 0.8199
Sensitivity (Recall): 0.7530
Specificity: 0.8869

Fold 4
Accuracy: 0.8199
Sensitivity (Recall): 0.7917
Specificity: 0.8482

Fold 5
Accuracy: 0.8110
Sensitivity (Recall): 0.7292
Specificity: 0.8929

=== Overall Metrics ===
Mean Accuracy: 0.8176
Mean Sensitivity: 0.7524
Mean Specificity: 0.8827


(0.8175595238095239, 0.7523809523809524, 0.8827380952380952)

In [169]:
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

NameError: name 'fpr' is not defined